## parallel_in_python3.ipynb

In [13]:
import sys
import os
from glob import glob
from time import time
import numpy as np
import pandas as pd
import cmlreaders
from cmlreaders import CMLReader

from cluster_helper.cluster import cluster_view
sys.path.insert(0, '/home1/dscho/code/projects/theta_reinstatement')
import pal_reinstatement as palr
sys.path.append('/home1/dscho/code/general')
import data_io as dio 

In [28]:
def run_sess_parallel(info):
    import sys
    sys.path.insert(0, '/home1/dscho/code/projects/theta_reinstatement')
    import pal_reinstatement as palr
    
    subj, sess, list_ = info
    try:  
        event_pairs = palr.run_sess(subj, 
                                    sess, 
                                    list_,
                                    overwrite=False,
                                    sleep_max=10)
        return None
    except:
        err = sys.exc_info()
        errf = '/home1/dscho/logs/TryExceptError-run_sess_parallel-{}-ses{}'.format(subj, sess)
        with open(errf, 'w') as f:
            f.write(str(err)+'\n')
        return None

# Power reinstatement matrices

In [15]:
# ix = CMLReader.get_data_index('r1')
ix = dio.open_pickle('/home1/dscho/projects/theta_reinstatement/data/metadata/r1_df.pkl')
expmt = 'PAL1'
ix = ix.query("(experiment=='{}')".format(expmt))
sessions = ix.apply(lambda x: (x['subject'], x['session']), axis=1).tolist()
print(len(sessions))

151


In [17]:
recall_pct_thresh = 1/6
recall_n_thresh = 20

info_list = []
n_sess = 0
for row in sessions:
    subj, sess = row
    reader = CMLReader(subj, expmt, sess)
    events, event_stats = palr.reader_load_events(reader, verbose=False)
    
    # Check inclusion criteria upfront.
    if (event_stats['pct_correct'] >= recall_pct_thresh) & (event_stats['n_correct'] >= recall_n_thresh):
        n_sess += 1
        lists = np.array([i for i in pd.unique(events['list']) if (i>0)])
        for list_ in lists:
            info_list.append((subj, sess, list_))
            
print('Will process {} lists from {} subjects over {} sessions'
      .format(len(info_list), len(np.unique(np.array(info_list)[:, 0])), n_sess))

Will process 2594 lists from 53 subjects over 112 sessions


In [29]:
# Parallel processing
n_jobs = len(ix)
print('Running code for {} jobs.\n'.format(n_jobs))
with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=120, cores_per_job=1) as view:
    output = view.map(run_sess_parallel, info_list)

Running code for 151 jobs.

30 Engines running
Sending a shutdown signal to the controller and engines.


OSError: [Errno 16] Device or resource busy: '.nfs000000110001c20800000863'